In [ ]:
import requests
import urllib.request
import json

from SPARQLWrapper import SPARQLWrapper, JSON


## Wikidata

### Recherche des auteurs dans Wikidata

Les classes potentiellement pertinentes dans Wikidata sont les suivantes:

* wd:Q36180 Écrivain
    * wd:Q49757	    poète
    * wd:Q214917	dramaturge
    * wd:Q381353	femme de lettres
    * wd:Q6625963	romancier
    * wd:Q4853732	auteur de littérature pour la jeunesse
    * wd:Q5434338	écrivain de fantasy
    * wd:Q1626130	homme de lettres
    * wd:Q10297252	auteur de roman policier
    * wd:Q18844224	écrivain de science-fiction
    * wd:Q15980158	essayiste
    * wd:Q26203955	auteur de contes
    * wd:Q11774202  essayiste
    * wd:Q1930187   jounraliste
    * wd:Q487596    dramaturge
    * wd:Q201788    historien


Dans une expérience préliminaire nous avons tenté de retrouver les personnes qui sont des instances de ces classes, nées au Canada et dont la langue est le français.

Si on prend la liste des écrivains qu'on a ainsi extraite, et qu'on analyse les écrivains  qui ne se retrouvent pas dans DBpdedia, on peut faire les constatations suivantes:

* Près du quart sont effectivement des écrivains québécois (les autres sont en général des auteurs hors Québec, des universitaires, ou des gens qui ne sont tout simplement pas écrivains)
* Parmi les écrivains québécois, leur absence de la liste peut être expliquée par les raisons suivantes:
    * Erreur dans l'écriture du nom, faisant en sorte que le nom ne correspond pas à l'URI dans DBpedia (Marc-André au lieu de Marc André, par exemple)
    * La personne est présente dans Wikipedia mais pas dans DBpedia
    * Dans la plupart des cas, les catégories associées à la personne n'ont pas permis de l'identifier clairement comme écrivain
* Parmi les écrivains québécois, vraiment très peu sont membres de l'UNEQ (ce qui signifie que cela ne nous aiderait pas à les détecter)


Il serait risqué d'utiliser cette liste extraite de Wikidata, puisqu'on aura environ 4000 instances qui ne sont pas des écrivains québécois.

On va donc plutôt faire le contraire: on va tenter de prendre les écrivains que nous avons extrait de DBpedia et les retrouver dans Wikidata.

In [ ]:
from time import sleep

f_dbpedia_fr = open('../Data/DBpedia/ecrivains_dbpedia_fr.txt')
f_dbpedia_en = open('../Data/DBpedia/ecrivains_dbpedia_fr.txt')

ecrivains = set()
for l in f_dbpedia_fr.readlines():
    (_,nom) = l.strip().split(' ; ')
    ecrivains.add(nom)
f_dbpedia_fr.close()

for l in f_dbpedia_en.readlines():
    (_,nom) = l.strip().split(' ; ')
    ecrivains.add(nom)
f_dbpedia_en.close()

    




q_ecrivain = """
SELECT  DISTINCT ?s 
WHERE 
{{

  {{ ?s rdfs:label "{0}"@fr }} UNION {{ ?s rdfs:label "{0}"@en }} UNION {{ ?s skos:altLabel "{0}"@fr }}
  
  {{ ?s wdt:P106  wd:Q36180 }} UNION    
  {{ ?s wdt:P106  wd:Q49757 }} UNION
  {{ ?s wdt:P106  wd:Q214917 }} UNION
  {{ ?s wdt:P106  wd:Q381353 }} UNION
  {{ ?s wdt:P106  wd:Q6625963 }} UNION
  {{ ?s wdt:P106  wd:Q4853732 }} UNION
  {{ ?s wdt:P106  wd:Q5434338 }} UNION
  {{ ?s wdt:P106  wd:Q1626130 }} UNION
  {{ ?s wdt:P106  wd:Q10297252 }} UNION 
  {{ ?s wdt:P106  wd:Q18844224 }} UNION 
  {{ ?s wdt:P106  wd:Q15980158 }} UNION 
  {{ ?s wdt:P106  wd:Q26203955 }} UNION
  {{ ?s wdt:P106  wd:Q11774202 }} UNION
  {{ ?s wdt:P106  wd:Q1930187 }} UNION
  {{ ?s wdt:P106  wd:Q487596 }} UNION
  {{ ?s wdt:P106  wd:Q201788 }}

}}
"""

ecrivains_wikidata = []

for e in ecrivains:
    sleep(1)
    print("---" + e + "----")
    rep_wikidata_ecrivain = requests.get('https://query.wikidata.org/sparql?format=json&query=' + q_ecrivain.format(e))
    if rep_wikidata_ecrivain.status_code == 200:
        if len(rep_wikidata_ecrivain.json()['results']['bindings']) > 0:
            for resultat in rep_wikidata_ecrivain.json()['results']['bindings']:
                ecrivains_wikidata.append((e, resultat['s']['value']))
                print(resultat['s'])
        else:
            print("PAS TROUVÉ DANS WIKIDATA")
    else:
        print('ERREUR:',e)


In [ ]:
print("Sur les",len(ecrivains), 'trouvés dans DBpedia, on en retrouve',len(ecrivains_wikidata), "dans Wikidata")

In [ ]:
f = open('../Data/Wikidata/ecrivains_wikidata2.txt','w')
for (nom,uri) in ecrivains_wikidata:
    f.write("{}<>{}\n".format(nom,uri))
f.close()